В данных 54 магазина и 33 семейства продуктов.                     
Временной ряд тренировочных данных с 01.01.2013 по 15.08.2017.              
Временной ряд тестовых данных для submition составляет 16 дней после последней даты тренировочных данных: с 16.08.2017 по 31.08.2017.                   
                                 
Нужно составить прогноз продаж для каждого из семейств продуктов в каждом из магазинов.     
                             
В отдельном ноутбуке проведен Feature engineering и сформированы необходимые датасеты.
                                
**Текущие датасеты**                   
1. final_featured_data - объединенные данные test и train с новыми признаками, отобранными по значимости.
2. zero_prediction - данные тех товаров, которые не продавались в конкретном магазине с начала 2013 года, исходя из чего можно предположить, что данные товары не будут продаваться в ближайшие 16 дней. Этот датасет мы будем объединять с предсказанными данными перед отправкой в submit.           
3. fuller_not_stat - не стационарные по результатам подсчета критерия Дикки-Фуллера ряды из числа рядов "магазин+семейство товаров"

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import gc
from tqdm.auto import tqdm

In [ ]:
from fbprophet import Prophet

# Import Data

In [ ]:
# Import
df = pd.read_csv("../input/featured-data-with-imp-feat/final_featured_data.csv")
zero_prediction = pd.read_csv("../input/zero-prediction/zero_prediction.csv")
fuller_result = pd.read_csv("../input/fuller-result/fuller_not_stat.csv")

# Datetime
df["date"] = pd.to_datetime(df.date)

zero_prediction = zero_prediction.set_index(['store_nbr', 'family', 'date']).sort_index()
d_train = df[df.date<'2017-08-16'].copy()
d_test = df[df.date>='2017-08-16'].copy()

In [ ]:
d_train.sample(2)

In [ ]:
d_train.info()

In [ ]:
# Переименовываем столбцы для Prophet
d_train.columns = ['family', 'store_nbr', 'ds', 'id', 'y', 'season', 'quarter',
       'week_of_month', 'year', 'day_of_week', 'month', 'day_of_month',
       'onpromotion', 'week_of_year', 'day_of_year']

In [ ]:
d_test[(d_test.store_nbr == 1) & (d_test.family == 'SEAFOOD')].onpromotion

# Метрики

In [ ]:
def compute_metrics(real, forecast):
    result = {}
    real=np.array(real)
    forecast=np.array(forecast)
    result['MSE'] = round(((real-forecast)**2).mean(),4)
    result['RMSE'] = round((((real-forecast)**2)**(1/2)).mean(),4)
    mape_list = []
    for i in range(len(real)):
        if real[i] == 0:
            mape_list.append(0)
        else:
            mape_list.append(abs(real[i]-forecast[i])/real[i])
        
    result['MAPE'] = round(np.mean(mape_list),4)
    result['SMAP'] = round(2.0 * np.mean(np.abs(forecast - real) / (np.abs(forecast) + np.abs(real))),4)
    return pd.Series(result)

# Validation

In [ ]:
# Возьмём дни часть данных для просчета внутренних метрик
valid_peiod= 15

# Prophet for one ts

Возьмем один временной ряд (в одном магазине по одному из семейств товаров) и посчитаем, как меняется внутренние метрики на этом ряду.

In [ ]:
# simple prophit
model = Prophet()
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y']].copy()
train = f_train[:-valid_peiod].copy()
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
fig = model.plot(forecast)

metrics:                           
MSE     5.8709                      
RMSE    1.9949                      
MAPE    0.7771                      
SMAP    0.4939

In [ ]:
# логистический prophit с верхней и нижней границей
model = Prophet(growth='logistic')
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y']].copy()
train = f_train[:-valid_peiod].copy()
train['cap'] = 20
train['floor'] = 0

model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
future['cap'] = 20
future['floor'] = 0
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
fig = model.plot(forecast)

metrics:                              
MSE     5.8948                             
RMSE    2.1312                         
MAPE    0.9551                         
SMAP    0.5123                           
Судя по метрикам, прогноз ухудшился

In [ ]:
# изменение гибкости тренда prophit (по умолчанию changepoint_prior_scale=0.05)
model = Prophet(changepoint_prior_scale=0.5)
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y']].copy()
train = f_train[:-valid_peiod].copy()
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
fig = model.plot(forecast)

metrics:                              
MSE     5.8787                       
RMSE    1.9948                        
MAPE    0.7765                     
SMAP    0.4940                      
Увеличение гибкости не дает прироста относительно простой модели.

In [ ]:
holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")
holidays["date"] = pd.to_datetime(holidays.date)
holidays.sample(3)

In [ ]:
holidays_pf = holidays[['date', 'type']].copy()
holidays_pf.columns = ['ds', 'holiday']

In [ ]:
holidays_pf

In [ ]:
# prophit with holidays
model = Prophet(holidays=holidays_pf)
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y']].copy()
train = f_train[:-valid_peiod].copy()
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
fig = model.plot_components(forecast)

metrics:                           
MSE     5.6498                   
RMSE    1.9843                        
MAPE    0.7752                     
SMAP    0.4935                          
Незначительное улучшение MSE и RMSE по сравнению с simple prophet.

In [ ]:
# добавление признаков в prophit
model = Prophet()
model.add_regressor('season')
model.add_regressor('quarter')
model.add_regressor('week_of_month')
model.add_regressor('year')
model.add_regressor('day_of_week')
model.add_regressor('month')
model.add_regressor('day_of_month')
model.add_regressor('onpromotion')
model.add_regressor('week_of_year')
model.add_regressor('day_of_year')
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y','season', 'quarter',
       'week_of_month', 'year', 'day_of_week', 'month', 'day_of_month',
       'onpromotion', 'week_of_year', 'day_of_year']].copy()
train = f_train[:-valid_peiod].copy()
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod+16)
future[['season', 'quarter',
       'week_of_month', 'year', 'day_of_week', 'month', 'day_of_month',
       'onpromotion', 'week_of_year', 'day_of_year']]= d_train[['season', 'quarter',
       'week_of_month', 'year', 'day_of_week', 'month', 'day_of_month',
       'onpromotion', 'week_of_year', 'day_of_year']].copy()
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
fig = model.plot_components(forecast)

metrics:                    
MSE     5.7520             
RMSE    2.0447             
MAPE    0.8117            
SMAP    0.5022                
Наблюдается ухудшение метрик на контрольном участке.

In [ ]:
# prophet с добавлением сезонности по условию выходных дней, 
# месячной сезонности и дополнительного параметра onpromotion
model = Prophet(weekly_seasonality=False)
model.add_seasonality(name='weekend', period=7, fourier_order=3, condition_name='is_wknd')
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.add_regressor('onpromotion')
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y','onpromotion']].copy()
train = f_train[:-valid_peiod].copy()
train["is_wknd"] = (train.ds.dt.weekday // 5).astype("int8")
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
future['is_wknd'] = (future.ds.dt.weekday // 5).astype("int8")
future['onpromotion']= d_train['onpromotion'].copy()
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
# fig = model.plot(forecast)

metrics:                        
MSE     5.4063                     
RMSE    1.9111                        
MAPE    0.7682                   
SMAP    0.4750                                    
Наблюдаем незначительное улучшение метрик по сравнению с simple prophet                

Итак, улучшения по сравнению с simple prophet на тестовом примере были с добавлением holidays и с заменой недельной сезонности по условию выходных дней, месячной сезонностью и доп.признаком onpromotion.                       
Попробуем объединить два решения.

In [ ]:
# prophet с добавлением сезонности по условию выходных дней, 
# месячной сезонности и дополнительного параметра onpromotion
# добавлены holidays
model = Prophet(holidays=holidays_pf, weekly_seasonality=False)
model.add_seasonality(name='weekend', period=7, fourier_order=3, condition_name='is_wknd')
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.add_regressor('onpromotion')
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y','onpromotion']].copy()
train = f_train[:-valid_peiod].copy()
train["is_wknd"] = (train.ds.dt.weekday // 5).astype("int8")
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
future['is_wknd'] = (future.ds.dt.weekday // 5).astype("int8")
future['onpromotion']= d_train['onpromotion'].copy()
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
# fig = model.plot(forecast)

metrics:                          
MSE     5.2253                       
RMSE    1.8792                        
MAPE    0.7759                     
SMAP    0.4691                         
Метрики улучшились

In [ ]:
del df, d_train, d_test

In [ ]:
gc.collect()

In [ ]:
# Попробуем добавить признак oil_over_70 из датасета стоимости нефти.
# Для этого нам нужен другой предобработанный датасет со всеми сгенерированными фичами
df = pd.read_csv("../input/featured-data/featured_data.csv")
# Datetime
df["date"] = pd.to_datetime(df.date)
df.columns = ['family', 'store_nbr', 'ds', 'id', 'y', 'onpromotion',
       'day_of_week', 'month', 'year', 'oil_over_70', 'is_active_family',
       'city', 'state', 'type', 'cluster', 'events_Black_Friday',
       'events_Cyber_Monday', 'events_Dia_de_la_Madre', 'events_Futbol',
       'events_Terremoto_Manabi', 'holiday_national_binary',
       'holiday_local_binary', 'holiday_regional_binary',
       'national_independence', 'local_cantonizacio', 'local_fundacion',
       'local_independencia', 'holiday_national_Batalla_de_Pichincha',
       'holiday_national_Carnaval', 'holiday_national_Dia_de_Difuntos',
       'holiday_national_Dia_de_la_Madre', 'holiday_national_Dia_del_Trabajo',
       'holiday_national_Independencia_de_Cuenca',
       'holiday_national_Independencia_de_Guayaquil',
       'holiday_national_Navidad',
       'holiday_national_Primer_Grito_de_Independencia',
       'holiday_national_Primer_dia_del_ano', 'holiday_national_Viernes_Santo',
       'holiday_regional_Provincializacion_Santa_Elena',
       'holiday_regional_Provincializacion_de_Cotopaxi',
       'holiday_regional_Provincializacion_de_Imbabura',
       'holiday_regional_Provincializacion_de_Santo_Domingo',
       'holiday_local_Cantonizacion_de_Cayambe',
       'holiday_local_Cantonizacion_de_El_Carmen',
       'holiday_local_Cantonizacion_de_Guaranda',
       'holiday_local_Cantonizacion_de_Latacunga',
       'holiday_local_Cantonizacion_de_Libertad',
       'holiday_local_Cantonizacion_de_Quevedo',
       'holiday_local_Cantonizacion_de_Riobamba',
       'holiday_local_Cantonizacion_de_Salinas',
       'holiday_local_Cantonizacion_del_Puyo',
       'holiday_local_Fundacion_de_Ambato',
       'holiday_local_Fundacion_de_Cuenca',
       'holiday_local_Fundacion_de_Esmeraldas',
       'holiday_local_Fundacion_de_Guayaquil',
       'holiday_local_Fundacion_de_Ibarra', 'holiday_local_Fundacion_de_Loja',
       'holiday_local_Fundacion_de_Machala',
       'holiday_local_Fundacion_de_Manta', 'holiday_local_Fundacion_de_Quito',
       'holiday_local_Fundacion_de_Riobamba',
       'holiday_local_Fundacion_de_Santo_Domingo',
       'holiday_local_Independencia_de_Ambato',
       'holiday_local_Independencia_de_Guaranda',
       'holiday_local_Independencia_de_Latacunga', 'day_of_month',
       'day_of_year', 'week_of_month', 'week_of_year', 'is_wknd', 'quarter',
       'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end',
       'is_year_start', 'is_year_end', 'season', 'workday', 'wageday']

d_train = df[df.ds<'2017-08-16'].copy()
d_test = df[df.ds>='2017-08-16'].copy()

In [ ]:
# prophet с добавлением сезонности по условию выходных дней, 
# месячной сезонности и дополнительного параметра onpromotion
# добавлены holidays и oil_over_70
model = Prophet(holidays=holidays_pf, weekly_seasonality=False)
model.add_seasonality(name='weekend', period=7, fourier_order=3, condition_name='is_wknd')
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.add_regressor('onpromotion')
model.add_regressor('oil_over_70')
f_train = d_train.loc[(d_train.family == 'AUTOMOTIVE') & (d_train.store_nbr == 1),['ds','y','onpromotion', 'oil_over_70']].copy()
train = f_train[:-valid_peiod].copy()
train["is_wknd"] = (train.ds.dt.weekday // 5).astype("int8")
model.fit(train)
future = model.make_future_dataframe(periods=valid_peiod)
future['is_wknd'] = (future.ds.dt.weekday // 5).astype("int8")
future[['onpromotion', 'oil_over_70']]= d_train[['onpromotion', 'oil_over_70']].copy()
forecast = model.predict(future)
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
print('metrics: \n', metrics)
# fig = model.plot(forecast)

metrics:                          
MSE     5.2194                         
RMSE    1.8735                         
MAPE    0.7652                 
SMAP    0.4683                           
Метрики улучшились. Пробуем просчитать этот вариант на общем датасете.

In [ ]:
del df
gc.collect()

# Prophet with holidays, onpromotion and oil price features

In [ ]:
metrics_by_store_fam = pd.DataFrame(columns=['store', 'family', 'MSE', 'RMSE', 'MAPE', 'SMAP'])

In [ ]:

for num, store in enumerate(tqdm(d_train.store_nbr.unique())):
    for numf, fam in enumerate(tqdm(d_train.family.unique())):
        f_train = d_train.loc[(d_train.family == fam) & (d_train.store_nbr == store),['ds','y','onpromotion', 'oil_over_70']].copy()
        train = f_train[:-valid_peiod].copy()
        train["is_wknd"] = (train.ds.dt.weekday // 5).astype("int8")
        if len(train)!=0:
            model = Prophet(holidays=holidays_pf, weekly_seasonality=False)
            model.add_seasonality(name='weekend', period=7, fourier_order=3, condition_name='is_wknd')
            model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
            model.add_regressor('onpromotion')
            model.add_regressor('oil_over_70')
            model.fit(train)
            future = model.make_future_dataframe(periods=(valid_peiod+16))
            future['is_wknd'] = (future.ds.dt.weekday // 5).astype("int8")
            future[['onpromotion', 'oil_over_70']]= d_train[['onpromotion', 'oil_over_70']].copy()
            forecast = model.predict(future)
            cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(f_train.set_index('ds'))
            metrics = compute_metrics(cmp_df.loc['2017-08-01':'2017-08-15','y'], cmp_df.loc['2017-08-01':'2017-08-15','yhat'])
            metric_dict = {
                'store': store,
                'family': fam,
                'MSE': metrics['MSE'],
                'RMSE': metrics['RMSE'],
                'MAPE': metrics['MAPE'],
                'SMAP': metrics['SMAP'],
                }
            #metric_temp = pd.DataFrame(metric_dict, index=[1])
            metrics_by_store_fam = metrics_by_store_fam.append(metric_dict, ignore_index=True)
            d_test.loc[(d_test.family == fam) & (d_test.store_nbr == store), 'prophet'] = np.array(cmp_df.loc['2017-08-16':'2017-08-31', 'yhat'])
            del train, future, cmp_df, metrics, metric_dict
            gc.collect()
        
        
        
all_metrics = {
    'MSE': metrics_by_store_fam['MSE'].mean(),
    'RMSE': metrics_by_store_fam['RMSE'].mean(),
    'MAPE': metrics_by_store_fam['MAPE'].mean(),
    'SMAP': metrics_by_store_fam['SMAP'].mean()
}

In [ ]:
all_metrics

# Submit

In [ ]:
#d_test = d_test.reset_index().set_index(['store_nbr', 'family', 'date']).sort_index()
d_test = d_test.set_index(['store_nbr', 'family', 'ds']).sort_index()
for i in zero_prediction.index:
    d_test.loc[i,'prophet']=0

In [ ]:
d_test=d_test.reset_index()
prophet_submit = d_test[['id','prophet']].copy()
prophet_submit.columns = ['id', 'sales']
prophet_submit.to_csv('submission_prophet_holidays_onpromotion_oil.csv', index=False)

In [ ]:
prophet_submit.tail(5)

# Результаты опробованных модификаций prophet

Simple prophet:                                      
'MSE': 80409.39740370169,                       
'RMSE': 84.2245773279352,                
'MAPE': 0.38141578947368504,                  
'SMAP': 0.5482279352226718                
                
**Результат на kaggle 0.54588** 

prophet with holidays                                     
'MSE': 82528.27533880848,                              
'RMSE': 86.2086423944478,                                 
'MAPE': 0.3870958357432043,                         
'SMAP': 0.5512633892423366                                   
**Результат на kaggle 0.54765**

Prophet with holidays, onpromotion and oil price features              
'MSE': 96477.30711798726,            
'RMSE': 94.08131758241767,                 
'MAPE': 0.3820305957200698,                             
'SMAP': 0.5906130133024874              
Результаты ухудшились, submit не проводился.